In [ ]:
!pip install gradio pandas matplotlib

In [ ]:

# Versión adaptada para Gradio
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from datetime import datetime

# === Cargar stock inicial ===
def cargar_stock():
    data = {
        "Codigo": [1001, 2001, 3001, 4001, 5001],
        "Producto": ['Pantalon', 'Jean', 'Camisa', 'Blusa', 'Short'],
        "Cantidad": [10, 15, 20, 8, 5],
        "Precio": [55000, 70000, 35000, 45000, 50000]
    }
    return pd.DataFrame(data)

df_stock = cargar_stock()
df_ventas = pd.DataFrame(columns=["Fecha", "Producto", "Cantidad", "Total"])

# === Funciones gradio ===
def ver_stock():
    alerta = ""
    bajos = df_stock[df_stock["Cantidad"] <= 5]
    if not bajos.empty:
        alerta = "\n\n🚨 Productos con stock bajo:\n" + bajos.to_string(index=False)
    return df_stock, alerta


def vender(producto, cantidad):
    global df_stock, df_ventas

    if producto not in df_stock["Producto"].values:
        return "Producto no encontrado."

    fila = df_stock[df_stock["Producto"] == producto]
    stock_disp = int(fila["Cantidad"].values[0])
    precio = int(fila["Precio"].values[0])

    if cantidad > stock_disp:
        return "No hay suficiente stock disponible."

    subtotal = cantidad * precio
    df_stock.loc[df_stock["Producto"] == producto, "Cantidad"] -= cantidad

    fecha_actual = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    df_ventas.loc[len(df_ventas)] = [fecha_actual, producto, cantidad, subtotal]

    factura = f"""
🧾 FACTURA
Fecha: {fecha_actual}
Producto: {producto}
Cantidad: {cantidad}
Precio Unitario: ${precio:,.0f}
Subtotal: ${subtotal:,.0f}
"""
    return factura


def reporte():
    if df_ventas.empty:
        return "Aún no hay ventas registradas."
    return df_ventas


def grafica():
    if df_ventas.empty:
        return "No hay datos para graficar."

    resumen = df_ventas.groupby("Producto")["Cantidad"].sum()
    fig = plt.figure()
    resumen.plot(kind="bar", color="purple")
    plt.title("Productos más vendidos")
    plt.xlabel("Producto")
    plt.ylabel("Cantidad Vendida")
    plt.grid(True)
    return fig


def agregar_producto(codigo, producto, cantidad, precio):
    global df_stock
    producto = producto.strip().title()
    existe = df_stock["Producto"].str.lower().eq(producto.lower()).any()

    if existe:
        df_stock.loc[df_stock["Producto"].str.lower() == producto.lower(), "Cantidad"] += cantidad
        df_stock.loc[df_stock["Producto"].str.lower() == producto.lower(), "Precio"] = precio
        return f"🔄 Producto '{producto}' actualizado."
    else:
        nuevo = {"Codigo": codigo, "Producto": producto, "Cantidad": cantidad, "Precio": precio}
        df_stock.loc[len(df_stock)] = nuevo
        return f"✅ Producto '{producto}' agregado."

# === Interfaces gradio ===
interfaz_menu = gr.TabbedInterface(
    [
        gr.Interface(fn=ver_stock, inputs=[], outputs=["dataframe", "text"], title="🛒 Ver Stock"),

        gr.Interface(fn=vender,
                     inputs=[
                         gr.Dropdown(choices=df_stock["Producto"].tolist(), label="Producto"),
                         gr.Number(label="Cantidad", precision=0)
                     ],
                     outputs="text", title="🛍️ Realizar Venta"),

        gr.Interface(fn=reporte, inputs=[], outputs="dataframe", title="📄 Ver Reporte de Ventas"),

        gr.Interface(fn=grafica, inputs=[], outputs="plot", title="📊 Gráfica de Ventas"),

        gr.Interface(fn=agregar_producto,
                     inputs=[
                         gr.Number(label="Código", precision=0),
                         gr.Text(label="Producto"),
                         gr.Number(label="Cantidad", precision=0),
                         gr.Number(label="Precio", precision=0)
                     ],
                     outputs="text", title="➕ Agregar/Actualizar Producto")
    ],
    tab_names=["Ver Stock", "Vender", "Reporte", "Gráfica", "Agregar Producto"]
)

interfaz_menu.launch()
